# Consigna NX - Challenge.

Cada una de las filas del archivo (indexada por SK_ID_CURR) corresponde a una persona distinta que nos pidió un préstamo, y la columna "TARGET" dice qué ocurrió con ese pedido en particular (es decir, tiene un 1 si su pago se atrasó , es decir, que entraron en default, y 0 si no lo hizo). La idea es que puedas hacer un proceso que permita predecir los clientes futuros que van a entrar en default sobre el préstamo.
El archivo "HomeCredit_columns_description - HomeCredit_columns_description.parquet" tiene la información disponible sobre cada una de las columnas de los otros archivos adjuntos. 

Para poder orientarte nuestra evaluación, vamos a hacer foco en el EDA y en el modelo predictivo, sin entrar en profundidad si la métrica esta tuneada o no.

# Modelo de clasificación

In [1]:
%reset -f

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import ceil
import seaborn as sns

In [3]:
df_app = pd.read_csv('/Users/santiagogimenez/Desktop/NX - Challenge/preprocessed_application.csv')
df_ins = pd.read_csv('/Users/santiagogimenez/Desktop/NX - Challenge/preprocessed_instalment.csv')
df_cre = pd.read_csv('/Users/santiagogimenez/Desktop/NX - Challenge/preprocessed_credit.csv')

In [4]:
df_app

,Unnamed: 0,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,0,100002,1,Cash loans,M,0,1,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,1,100003,0,Cash loans,F,0,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,100004,0,Revolving loans,M,1,1,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,100006,0,Cash loans,F,0,1,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,4,100007,0,Cash loans,M,0,1,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307504,307504,456251,0,Cash loans,M,0,0,157500.0,254700.0,27558.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307505,307505,456252,0,Cash loans,F,0,1,72000.0,269550.0,12001.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307506,307506,456253,0,Cash loans,F,0,1,153000.0,677664.0,29979.0,...,0,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0
307507,307507,456254,1,Cash loans,F,0,1,171000.0,370107.0,20205.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
df_ins

,Unnamed: 0,SK_ID_CURR,AMT_PAYMENT_INSTALMENT,DAYS_ENTRY_INSTALMENT
0,0,100001,1.000000,1.004767
1,1,100002,1.000000,1.131260
2,2,100003,1.000000,1.007302
3,3,100004,1.000000,1.010037
4,4,100005,1.000000,1.039191
...,...,...,...,...
339582,339582,456251,1.000000,1.357098
339583,339583,456252,1.000000,1.001208
339584,339584,456253,0.928571,1.007221
339585,339585,456254,1.000000,1.293806


In [6]:
df_cre

,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_TOTAL_DRAWINGS,AMT_PAYMENT_TOTAL_CURRENT,CNT_TOTAL_DRAWINGS,CNT_INSTALMENT_MATURE_CUM,SK_DPD,SK_DPD_DEF
0,100006,-3.5,0.000000,270000.000000,NaN,0.000000,NaN,0.000000,0.000000,0.000000
1,100011,-38.5,54482.111149,164189.189189,4864.864865,4520.067568,0.108108,25.767123,0.000000,0.000000
2,100013,-48.5,18159.919219,131718.750000,12700.000000,6817.172344,0.511111,18.719101,0.010417,0.010417
3,100021,-10.0,0.000000,675000.000000,NaN,0.000000,NaN,0.000000,0.000000,0.000000
4,100023,-7.5,0.000000,135000.000000,NaN,0.000000,NaN,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
103553,456244,-21.0,131834.730732,296341.463415,53681.013659,32720.544878,2.731707,13.600000,0.000000,0.000000
103554,456246,-5.5,13136.731875,135000.000000,30398.512500,15554.340000,5.000000,3.500000,0.000000,0.000000
103555,456247,-49.0,23216.396211,144000.000000,4299.012947,4115.878105,0.294737,26.494737,0.031579,0.021053
103556,456248,-13.0,0.000000,900000.000000,NaN,0.000000,NaN,0.000000,0.000000,0.000000
